In [3]:
def calculate_proportions(input_values, output_values, manual_proportions):
    """
    We are using the haircut method to proportionally distribute input values
    to output values. But sometimes it is also important that manual proportions
    can be set.

    This function takes in a transaction and a list of manual input-to-output
    proportions and returns a list of input-to-output values.

    The output format will be an adjacency matrix specifying the values
    for each input-to-output pair. The input and output indices are the
    indices of the input_values and output_values lists.

    For example, if we have an input at index 0 with value 5, and an output
    at index 2 with value 6, and we send 0.5 of the input to the output, then
    the output of this function will include 2.5 at index (0, 2).

    This process is tricky since we have to force certain input-to-output
    pairs to have certain proportions, while also maintaining the haircut
    distributions. For instances, if we send 0.5 of an input to an output,
    the haircut method will try to force more from that input to the output.
    So we need to re-distribute the remainder of this input-to-output
    amount to the other outputs.

    It is easy to check the correctness of the final values. The sum of the
    final values for each input-to-output pair should equal the sum
    of the output values. And the values for the manual proportions should
    be as specified.
    """
    result = [[0 for _ in output_values] for _ in input_values]
    remaining_input_values = input_values.copy()
    remaining_output_values = output_values.copy()

    # First, we distribute the manual proportions
    # If any proportion is greater than 1, or causes the input value to
    # be greater than the output value, then we raise an error
    for i, o, p in manual_proportions:
        result[i][o] = input_values[i] * p
        remaining_input_values[i] -= result[i][o]
        remaining_output_values[o] -= result[i][o]
        if p > 1:
            raise ValueError("Manual proportion cannot be greater than 1")
        if result[i][o] > input_values[i]:
            raise ValueError("Manual proportion cannot be greater than input value")
        if result[i][o] > output_values[o]:
            raise ValueError("Manual proportion cannot be greater than output value")

    # collect together all non-manual edges, and sum their values
    non_manual_edges = []
    non_manual_sum = 0

    remaining_values_sum = sum(remaining_input_values)
    if remaining_values_sum > 0:
        # Next, distribute the remaining values proportionally. But when we
        # reach an input-to-output pair that has a manual proportion, we
        # need to distribute the remaining values to the other outputs.
        # This is done by first skipping these edges, then calculating the
        # remaining values and distributing them later.
        amount_remaining = 0
        for o, output_value in enumerate(remaining_output_values):
            for i, input_value in enumerate(remaining_input_values):
                if result[i][o] > 0:
                    amount_remaining += output_value * input_value / remaining_values_sum
                    continue
                result[i][o] += output_value * input_value / remaining_values_sum
                non_manual_edges.append((i, o, result[i][o]))
                non_manual_sum += result[i][o]

        # Finally, distribute the remaining values to the other outputs
        # Value is attributed based on what proportion each non-manual edge
        # has of the total non-manual sum
        for i, o, _ in non_manual_edges:
            result[i][o] += amount_remaining * (result[i][o] / non_manual_sum)

    return result


input_values = [5, 4, 3, 6]
output_values = [4, 8, 6]


# format: (input index, output index, proportion fraction)
manual_proportions = [
    (0, 0, 0.4),
    (0, 1, 0.4),
    (1, 1, 0.5),
    (2, 2, 1/3)
]


edges = calculate_proportions(input_values, output_values, manual_proportions)
display(edges)

print(f"total edges sum: {sum([sum(row) for row in edges])} (desired is 18)")

assert sum([sum(row) for row in edges]) == sum(output_values)

assert all([edges[i][j] == input_values[i] * proportion for i, j, proportion in manual_proportions])

input_values = [10, 100, 1]
output_values = [1, 3, 2]

manual_proportions = [
    (0, 1, 1/20),
    (1, 1, 1/200)
]

edges = calculate_proportions(input_values, output_values, manual_proportions)
display(edges)

print(f"total edges sum: {sum([sum(row) for row in edges])} (desired is {sum(output_values)})")

assert (sum([sum(row) for row in edges]) - sum(output_values)) < 0.0000000000000009
assert all([edges[i][j] == input_values[i] * proportion for i, j, proportion in manual_proportions])

input_values = [1, 24]
output_values = [1, 24]

manual_proportions = [
    (0, 0, 1.0),
    (1, 1, 1.0)
]

edges = calculate_proportions(input_values, output_values, manual_proportions)

print(f"total edges sum: {sum([sum(row) for row in edges])} (desired is {sum(output_values)})")

assert (sum([sum(row) for row in edges]) - sum(output_values)) < 0.0000000000000009
assert all([edges[i][j] == input_values[i] * proportion for i, j, proportion in manual_proportions])

[[2.0, 2.0, 0.5670103092783505],
 [0.4536082474226804, 2.0, 1.134020618556701],
 [0.4536082474226804, 0.9072164948453608, 1.0],
 [1.3608247422680413, 2.7216494845360826, 3.402061855670103]]

total edges sum: 18.0 (desired is 18)


[[0.1430722891566265, 0.5, 0.286144578313253],
 [1.4984939759036147, 0.5, 2.9969879518072293],
 [0.015060240963855422, 0.030120481927710843, 0.030120481927710843]]

total edges sum: 6.000000000000001 (desired is 6)
total edges sum: 25.0 (desired is 25)
